### This notebook estimates the maximum number of electrons per pixel the spaceX satellites would produce.

In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import lsst.syseng.throughputs as st
from lsst.sims.photUtils import PhotometricParameters, Sed
import galsim
import matplotlib.pyplot as plt

### We use Eq (6) in LSE-40. With the baseline system as defined by syseng_throughput (including hardware & atmosphere), We can easily map a source magnitude to a source count. 

In [ ]:
# full_well for a 30s visit, in e-
full_well = 150e3  #typical for e2v sensors. ITL is typically 200k. We try to be conservative and use the smaller.
full_well2 = 100e3  #some sensors can be as low as 100K e-.
bias_offset = 45e3
bias_offset_subtract = 0 #According to Steve R. and Aaron R., this should not be a factor

## We first determine what fraction of the flux will be in the center/brightest pixel
#### A satellite moves at 0.5 deg per sec. It is 4m wide and 550km high.

In [ ]:
fwhm_extended = 4/550e3/np.pi*180*3600
#fwhm = np.sqrt(fwhm_extended**2+.4**2+fwhms**2) #need Tony to confirm whether we should use this fwhm_extended as the total
fwhm = fwhm_extended
print('FWHM = %.2f arcsec'%fwhm)

In [ ]:
#Assume it moves along pixel grid. The time its image center moves from one pixel center to the next is dt
dt = 0.2/(0.5*3600) # in seconds

#### Approximate it as a source which appears at one pixel center for dt, then disappears from there, and instantly appears at the next pixel center for dt. During each dt, the source deposit a flux profile that is a Gaussian with FWHM of 1.5 arcsec.
So we are just overlapping Gaussians whose centers are 0.2 arcsec apart. Considering that the FWHM is much larger than pixel size, the top of the Gaussians are pretty flat. So this should be a pretty good approximation.
This slightly overestimates the flux, because having the Gaussian moving across the pixel in dt would yield a smaller flux for the pixel, compared to when the center of the Gaussian is overlapped with the pixel center during dt.

In [ ]:
stamp_size = 101
pixel_scale = 0.2
psf = galsim.Gaussian(fwhm=fwhm)
img = galsim.ImageD(stamp_size, stamp_size, scale=pixel_scale)
psf = psf.withFlux(1) #unit flux
psf.drawImage(image=img)
ratio = sum(img.array[50,:])/np.sum(img.array)
plt.plot(img.array[50,:])
print(ratio)
plt.grid()

In [ ]:
#what if the trail is 45 deg to the pixel grid?
# dt will be 1.414 time longer, would that give a larger ratio?
print(sum(np.diag(img.array))*1.414)
# it is the same. Bingo. That is it.

### Baseline LSST system, as defined in syseng_throughput

In [ ]:
defaultDirs = st.setDefaultDirs()
hardware, system = st.buildHardwareAndSystem(defaultDirs)

### Default photometric parameters, as used in standard m5 calculations

In [ ]:
exptime=15 
nexp=2
readnoise=8.8 
othernoise=0 
darkcurrent=0.2
effarea=np.pi*(6.423/2*100)**2
X=1.0

# PhotometricParameters object for standard m5 calculations.
photParams_std = PhotometricParameters(exptime=exptime, nexp=nexp,
                                           gain=1.0, effarea=effarea, readnoise=readnoise,
                                           othernoise=othernoise, darkcurrent=darkcurrent)

### Let's make sure we can reproduce standard m5 results

In [ ]:
m5 = st.makeM5(hardware, system, darksky=None, 
                      exptime=exptime, nexp=nexp, readnoise=readnoise, othernoise=othernoise, darkcurrent=darkcurrent,
                      effarea=effarea, X=1.0)

In [ ]:
m5

### Set up the dataframe

In [ ]:
filterlist = ('u', 'g', 'r', 'i', 'z', 'y')
properties = ['SatLim']
d = pd.DataFrame(index=filterlist, columns=properties, dtype='float')

### Calculate the saturation limits

In [ ]:
# change exposure to dt
photParams_dt = PhotometricParameters(exptime=dt, nexp=1,
                                           gain=1.0, effarea=effarea, readnoise=readnoise,
                                           othernoise=othernoise, darkcurrent=darkcurrent)
key = 'SatLim'
for f in system:
    flatsource = Sed()
    flatsource.setFlatSED(wavelen_min=system[f].wavelen_min, wavelen_max=system[f].wavelen_max,
                              wavelen_step=system[f].wavelen_step)
    adu = flatsource.calcADU(system[f], photParams=photParams_dt)

    adu0 = adu*ratio
    #because setFlatSED() assumes a m=0 star
    # we use gain=1.0, so adu = number of e-
    # we also get rid of skycounts from each pixel. This has miminal effect on results
    d[key].loc[f] = np.log10(adu0/(full_well-m5.skyCounts.loc[f]-bias_offset_subtract))/2*5
    # The above can also be obtained this way, 
    # (but flatsource has to be re-initialized for each calculation)
    # Scale fnu so that adu0 is equal to full well.
    #flatsource.fnu = flatsource.fnu * (full_well/adu0)
    #d[key].loc[f] = flatsource.calcMag(system[f])

In [ ]:
d.join(m5)

#### Double-check: do the same calculation using zero points

In [ ]:
d_zp = pd.DataFrame(index=filterlist, columns=properties, dtype='float')
key = 'SatLim'
for f in system:
    #number of electrons we can accomodate for the source in dt sec
    ne = (full_well-m5.skyCounts[f]-bias_offset_subtract)/ratio
    #number of electrons we can accomodate for this source in 1 sec
    ne = ne/dt
    d_zp[key].loc[f] = (m5.Zp_t[f]-2.5*np.log10(ne))

In [ ]:
round(d_zp - d)  #should be idential to above calculations

### Make peak electron counts vs. mag plot

In [ ]:
pRatio = ratio
colors = ['blue', 'green', 'red', '0.75', '0.50', '0.25']
d_plot = pd.DataFrame(index=filterlist, columns=['SatLimFromPlot'], dtype='float')

In [ ]:
fig, ax = plt.subplots()
for i,f in enumerate(filterlist):
    mags = []
    npeaks = []
    flatsource = Sed()
    flatsource.setFlatSED(wavelen_min=system[f].wavelen_min, wavelen_max=system[f].wavelen_max,
                              wavelen_step=system[f].wavelen_step)
    adu = flatsource.calcADU(system[f], photParams=photParams_dt)
    adu0 = adu*pRatio #a m=0 start produces this adu0 in the center pixel
    for m in range(31):
        mag = 1 + 0.2 * m
        mags.append(mag)
        npeak = adu0*10**(-mag/2.5)+m5.skyCounts[f]+bias_offset_subtract
        npeaks.append(npeak)
    plt.scatter(mags, npeaks, label=f, color=colors[i])
    #print(npeak)
    #break
    d_plot.SatLimFromPlot.loc[f] = mags[np.argmax(np.array(npeaks)<full_well)]
plt.plot([0,7.0],[full_well, full_well], ls = '--', lw = 2, color='black')
plt.text(4.5, full_well*1.20, "Saturation = %d e-"%full_well, color = 'black')
plt.plot([0,7.0],[full_well2, full_well2], ls = '--', lw = 2, color='black')
plt.text(4.5, full_well2*0.7, "Saturation = %d e-"%full_well2, color = 'black')
plt.text(0.3, 0.24, "Trail FWHM = %.2f arcsec"%fwhm, color = 'black', transform=fig.transFigure)
plt.text(0.3, 0.18, "Dark Sky", color = 'black', transform=fig.transFigure)
plt.legend()
plt.yscale('log')
plt.xlim(0,7.0)
plt.ylim(1e3, 1.5e6)
plt.xlabel('Satellite apparent magnitude', fontsize=12)
plt.ylabel('Peak pixel count(electrons)', fontsize=12);

In [ ]:
#check consistency with above calculations.
d_plot.join(d)

### We can use the zeropoints to do the same calculations and make the same plot

In [ ]:
d_plot_zp = pd.DataFrame(index=filterlist, columns=['SatLimFromPlot'], dtype='float')
fig, ax = plt.subplots()
peak_e_mag_5 = []
for i,f in enumerate(filterlist):
    mags = []
    npeaks = []
    for m in range(31):
        mag = 1.0 + 0.2 * m
        mags.append(mag)
        #for a source of mag, how many electrons is produced on detector in a second?
        ne = 10**((m5.Zp_t[f] - mag)/2.5)
        # what about dt seconds
        ne *= dt
        # how many fall into the center pixel? on top of the background and others
        npeak = ne*pRatio+ m5.skyCounts[f]+bias_offset_subtract
        npeaks.append(npeak)
        d_plot_zp.SatLimFromPlot.loc[f] = mags[np.argmax(np.array(npeaks)<full_well)]
        if mag==5.0:
            peak_e_mag_5.append(npeak)
            print('for 5th mag, under dark sky, band %s, peak e per pixel = %.0f'%(f,npeak))
    plt.scatter(mags, npeaks, label=f, color=colors[i])
    #print(npeak)
    #break
plt.plot([0,7.0],[full_well, full_well], ls = '--', lw = 2, color='black')
plt.text(4.5, full_well*1.20, "Saturation = %d e-"%full_well, color = 'black')
plt.plot([0,7.0],[full_well2, full_well2], ls = '--', lw = 2, color='black')
plt.text(4.5, full_well2*0.7, "Saturation = %d e-"%full_well2, color = 'black')
plt.text(0.3, 0.24, "Trail FWHM = %.2f arcsec"%fwhm, color = 'black', transform=fig.transFigure)
plt.text(0.3, 0.18, "Dark Sky", color = 'black', transform=fig.transFigure)
plt.legend()
plt.yscale('log')
plt.xlim(0,7.0)
plt.ylim(1e3, 1.5e6)
plt.xlabel('Satellite apparent magnitude', fontsize=12)
plt.ylabel('Peak pixel count(electrons)', fontsize=12)
plt.savefig('Peak_count_by_band_trail_darkSky.pdf')

In [ ]:
round(d_plot - d_plot_zp) #should be identical to calculations using zero points

#### Now do the same thing for bright sky. We will only do the calculations using zero points below.

In [ ]:
import os
from lsst.utils import getPackageDir
def skyMag2Count(skyMag, f, hardware, photParams):
    '''
    inputs:
        skyMag is the sky magnitude
        f is the filter band
        hardware: an bandpass object as defined in the syseng_throughput package
        photParams: an PhotometricParameters object as defined in the syseng_throughput package
    output:
        skycount in each pixel
    '''
    darksky = Sed()
    darksky.readSED_flambda(os.path.join(getPackageDir('syseng_throughputs'),
                                             'siteProperties', 'darksky.dat'))
    fNorm = darksky.calcFluxNorm(skyMag, hardware)
    darksky.multiplyFluxNorm(fNorm)
    skyCount = (darksky.calcADU(hardware, photParams=photParams)* photParams.platescale**2)
    return fNorm, skyCount

skyBrightMag below is sky brightness at 50 deg from full moon (based on DeCam experience)

Tony wrote :

Even the 
11 yr solar cycle gives 0.5 mag change in v band.  For our immediate 
SpaceX purpose it would be appropriate to generate only two plots, one 
for bright time and one for dark time.  In the mid 2020s we enter solar 
max, so add 0.4 mag in v band to sky.

In [ ]:
skyBrightMag = {'u': 17.7, 'g':19.4, 'r':19.7, 'i':19.4, 'z':18.2, 'y':17.7}

In [ ]:
skyCounts = {}
for f in system:
    skyCount = 10**((0.4+m5.skyMag[f]-skyBrightMag[f])/2.5)*m5.skyCounts[f] # we can actually simply calculate sky count this way.
    skyCounts[f]=skyCount
    fNorm, skyCount1 = skyMag2Count(skyBrightMag[f]-0.4, f, hardware[f], photParams_std)
    print('%.2f, %.2f, %.2f'%(skyCount, skyCount1, skyCount-skyCount1)) # this is just to double check

In [ ]:
#use zero points to calculate saturation magnitudes under bright sky
d_zp_bright = pd.DataFrame(index=filterlist, columns=properties, dtype='float')
for f in system:
    key = 'SatLim'
    #number of electrons we can accomodate for the source in dt sec
    ne = (full_well-skyCounts[f]-bias_offset_subtract)/ratio
    #number of electrons we can accomodate for this source in 1 sec
    ne = ne/(dt)
    d_zp_bright[key].loc[f] = (m5.Zp_t[f]-2.5*np.log10(ne))

In [ ]:
d_zp_bright - d_zp

In [ ]:
d_plot_zp_bright = pd.DataFrame(index=filterlist, columns=['SatLimFromPlot'], dtype='float') # b for bright sky
fig, ax = plt.subplots()
for i,f in enumerate(filterlist):
    mags = []
    npeaks = []
    for m in range(31):
        mag = 1.0 + 0.2 * m
        mags.append(mag)
        #for a source of mag, how many electrons is produced on detector in a second?
        ne = 10**((m5.Zp_t[f] - mag)/2.5)
        # what about dt seconds
        ne *= dt
        # how many fall into the center pixel? on top of the background and others
        npeak = ne*pRatio+ skyCounts[f]+bias_offset_subtract
        npeaks.append(npeak)
        d_plot_zp_bright.SatLimFromPlot.loc[f] = mags[np.argmax(np.array(npeaks)<full_well)]
        if mag==5.0:
            peak_e_mag_5.append(npeak)
            print('for 5th mag, under bright sky, band %s, peak e per pixel = %.0f'%(f,npeak))
    plt.scatter(mags, npeaks, label=f, color=colors[i])
    #print(npeak)
    #break
plt.plot([0,7.0],[full_well, full_well], ls = '--', lw = 2, color='black')
plt.text(4.5, full_well*1.20, "Saturation = %d e-"%full_well, color = 'black')
plt.plot([0,7.0],[full_well2, full_well2], ls = '--', lw = 2, color='black')
plt.text(4.5, full_well2*0.7, "Saturation = %d e-"%full_well2, color = 'black')
plt.text(0.3, 0.24, "Trail FWHM = %.2f arcsec"%fwhm, color = 'black', transform=fig.transFigure)
plt.text(0.3, 0.18, "Bright Sky", color = 'black', transform=fig.transFigure)
plt.legend()
plt.yscale('log')
plt.xlim(0,7.0)
plt.ylim(1e3, 1.5e6)
plt.xlabel('Satellite apparent magnitude', fontsize=12)
plt.ylabel('Peak pixel count(electrons)', fontsize=12)
plt.savefig('Peak_count_by_band_trail_brightSky.pdf')

In [ ]:
d_plot_zp_bright - d_plot_zp